# Kalman Filter - Lab 7.2

## Recap
This is the Lab on using a Kalman Filter in CE6003's Object Tracking. You should complete the tasks in this lab as part of the Kalman Filter section of the lesson.

Please remember this lab must be completed before taking the quiz at the end of this lesson.

First, if we haven't already done so, we need to clone the various images and resources needed to run these labs into our workspace.

In [0]:
!git clone https://github.com/EmdaloTechnologies/CE6003.git




**Program Description**

This program demonstrates a very simple 'tracking' mechanism - derived from a Kalman filter. We're going to use our Kalman filter to track a single object, namely a person.


In [0]:
import os
import re
import io
import cv2
import time
import numpy as np
import base64
from IPython.display import clear_output, Image, display

#The Story So Far

To illustrate how to track something in a video stream, we have used the following technique to generate a set of images for you to work on.

What we did was we generated a short video - just recording one person walking around, on an iPhone.

Then we used ```ffmpeg``` to decompose about 7 seconds of that video down into still images.

```ffmpeg -i $vid_in -vf fps=30 imgs/daire%03d.png```

We saved those frames as ```imgs/daire%03d.png``` in the git repository in the single-detections directory

We've run yolo3 over those frames to generate bounding boxes and saved those bounding boxes into the same directory.

The file format is comma-separated values and the values are as shown here:

 frame index | object-type | centre-x | centre-y | width | height | confidence
 --- | --- | --- | --- | --- | --- | ---
 int | -1 | float | float | float | float | float

* The object type is always a person - that's all we inferred for.
* The centre-x and width are fractions of the image's width
* The centre-y and height are fractions of the image's height
* The confidence is supplied by Yolo3

*What Happens Now*

For each image in the directory, in order,
* we'll find the centre of the detection in that image (if any)
* we'll build a bounding box for the detection in that image
* we'' derive a variance term (crudely) from the Yolo confidence for that image
* and we'll supply the centre of that bounding box along with the variance term to a Kalman Filter implementation

Then, we'll explore how a Kalman filter tracks the object in the image stream.

**Get File Handles**

This function gets the filenames of all the files in the directory, in a reproducible order, and loads in the bounding boxes from file.

In [0]:
def get_pngs_and_boxes():
    pngdir = "/content/CE6003/images/lab7/single-objects/"
    bbdir = "/content/CE6003/images/lab7/single-objects/"

    pngfolder = os.fsencode(pngdir)
    bbfolder = os.fsencode(bbdir)

    pngfiles = []
    for filename in os.listdir(pngfolder):
        if filename.decode().endswith(".png"):
            pngfiles.append(pngdir + filename.decode())
    pngfiles.sort()

    for filename in os.listdir(bbfolder):
        if filename.decode().endswith(".boxes"):
            bbfilename = bbdir + filename.decode()

    bb = open(bbfilename, "r")
    bb_lines = bb.readlines()
    bb.close()

    return bb_lines, pngfiles

**Parse Detections**

We'll use this function in the main loop to wrangle the detections into the format we want to supply to our Kalman Filter.

Essentially it takes the name of png file, an img object and the list of bounding boxes as inputs.

It then finds the correct record (if any) for that image in the bounding boxes list and converts the bounding box parameters into a format which we'll use for the rest of the program (it converts back to absolute pixel values).

It returns a centre and a confidence value for the image supplied to it.

In [0]:
def parse_detections(bboxes, pngfile, img):
    # Sample Line: 400,-1,0.285417,0.241667,0.094792,0.483333,0.999797,-1,-1,-1
    # Index, object type,
    # x    - centre of bounding box (as fraction of image width
    # y    - centre of bounding box (as fraction of image height
    # w    - width of bounding box (as fraction of image width)
    # h    - height of bounding box (as fraction of image height
    # prob, _,_,_

    # extract the frame index of the png file - 
    # use it to find the detections for that frame
    index = int(re.findall(r'\d+', pngfile)[-1])
    imgh, imgw = img.shape[:2]

    centre = np.zeros(shape=(2, 1))
    P = 0.000001 # hack to avoid div by zero
    for line in bboxes:
        np_array = np.genfromtxt(io.StringIO(line), delimiter=",")
        lineindex = int(np_array[0])

        if lineindex == index:
            centre = np_array[2:4]
            P += np_array[6]
            centre[0] *= imgw
            centre[1] *= imgh

            return centre, P

    return centre, P

**Kalman 2D**

This function specialises the kalman() routine below for a particular model.

In this example, we are going to create a 2D Kalman, and we'll use a constant velocity model - so we'll use a term for x (expressed as position and velocity) and a term for y (also expressed as position and velocity).

### State Estimate Term
Therefore our state estimate will be represented by 4 state terms

$x_{state} = \begin{bmatrix}
    x_{pos} \\
    y_{pos} \\
    \dot{x} \\
    \dot{y} 
  \end{bmatrix}$

Note: the $x$ in $x_{state}$ is separate from $x_{pos}$.


### State Estimate Error Covariance
Our state estimate error covariance matrix will have a $4 \times 4$ shape.
We'll supply something like:

$P_{0} = \begin{bmatrix}
  100 &&   0 &&   0 &&   0 \\
  0   && 100 &&   0 &&   0 \\
  0   &&   0 && 100 &&   0 \\
  0   &&   0 &&   0 && 100 \\
  \end{bmatrix}$

to this matrix later on to initialize our state estimate error covariance, indicating a high initial degree of uncertainty around each of our state parameters and indicating an initial belief that the uncertainty in each state term is not linked to the uncertainty in the other state terms.

This is a measure of the initialial estimated accuracy of the state estimate $x_{state}$ above.

Its typical to initialize $P$ by setting the diagonal elements to the uncertainty in your initial values of the state and normally to quite a large value. 

The measurements will normally bring the state estimate error covariances down when running the filter and in almost all cases, the off diagonal elements will become non-zero as the filter works.

For the case of the simple 2 state position-velocity filter, the off diagonal matrix elements make sure that if you do have a velocity measurement, that the position is also corrected. So, the state estimate error covariance matrix 'learns' how variance in one state term affects the variance in the other state terms.

### Process Noise Covariance

We'll initialise the process noise covariance as an Identity matrix as shown here.  We're setting it up along the diagonal because this is a reasonable way to set up when we have no good information about the process.  You can better model this term (particularly for more advanced filters) but for our purposes it leads to satisfactory tracking.  Essentially we are saying that we believe each state term is effectively independent of each other state term.  It turns out that even in cases where this is patently not true, it leads to reasonable behaviour and obviates the need to a detailed process model - which suits our purposes in this demonstration.

$Q_{0} = \begin{bmatrix}
 1 && 0 && 0 && 0 \\
 0 && 1 && 0 && 0 \\
 0 && 0 && 1 && 0 \\
 0 && 0 && 0 && 1
 \end{bmatrix}$


### Measurement Terms

We'll supply a measurement term (aka an observation term) later on dynamically to the filter for each frame we process. In terms of the shape of the measurement term its going to represent what we can measure - i.e. $x_{pos}$ and $y_{pos}$. So, our measurement term will be of this form:

$measurement = \begin{bmatrix}
 x_{pos} \\
 y_{pos} \\
 \end{bmatrix}$

### Measurement uncertainty covariance
Again, we'll supply our measurement noise co-variance values (aka our observation nose covariance) later on dynamically, but, in terms of shape it will look like this.  We're assuming empirically a certain measurement noise in the two terms we're measuring $x_{pos}$ and $y_{pos}$.

$R = \begin{bmatrix}
  x_{noise} && 0 \\
  0 && y_{noise}
 \end{bmatrix}$

We'll derive empirically $x_{noise}$ and $y_{noise}$ from yolo later for each frame we process.

### State-Transition Model and Observation Model
Finally, we'll use two shape terms to tell a generic Kalman Filter algorithm what terms to multiply at each stage.

$F$, the state transition model, will effectively enforce the constant velocity model on the process.

$F =\begin{bmatrix}
  1 && 0 && 1 && 0 \\
  0 && 1 && 0 && 1 \\
  0 && 0 && 1 && 0 \\
  0 && 0 && 0 && 1
  \end{bmatrix}$

$H$, the observation model matrix, effectively informs the other matrices in the ```kalman()``` function below that we are only measuring $x_{pos}$ and $y_{pos}$

$H=\begin{bmatrix}
            1 && 0 && 0 && 0 \\
            0 && 1 && 0 && 0
 \end{bmatrix}$

### A note on 'Tuning' the Kalman Filter
Setting each term along the diagonal of the $Q$ matrix (the process error covariance term) to 1 works as a 'bucket chemistry' tuning technique for filter responsiveness when working in conjunction with estimates for the $R$ matrix (the observation covariance).  For some projects we have the time and resources to derive statistical models to accurately model $Q$ and $R$ but for some projects - such as this introductory example - its relatively common to tune them quickly by hand.

Some tuning tips include:
* Trial and error tuning is common;
* Setting large values of $Q$ relative to $R$ tells the Kalman Filter to trust the observations more than the model;
* setting large values of $R$ relative to $Q$ tells the Kalman Filter to trust the model more than the observations;
* setting $Q$ to all ones along the diagonal is reasonably common as its relatively neutral and means you can tune by only adjusting $R$.

### A note on types

Finally, we initialise everything using floats, to get a self-consistent set of data types - everything is floating point.

### Summary

With these matrices shaped and initialized, we have specialised a generic ```kalman()``` routine to perform Kalman Filtering using a constant velocity model in two dimensions; x and y.  For simplicity, we are not using $B_{k}$, the control-input model, in this example.

In [0]:
def kalman_2d(x, P, observation, R, Q=np.matrix(np.eye(4))):
    x, P = kalman(x, P, observation, R, Q,
        # State Transition Model matrix, assuming constant velocity model (x, y, x_dot, y_dot)
        F=np.matrix('''
            1. 0. 1. 0.;
            0. 1. 0. 1.;
            0. 0. 1. 0.;
            0. 0. 0. 1.
            '''),
        # Observation Model matrix, assuming we can only measure 
        # the co-ordinates x, y
        H=np.matrix('''
            1. 0. 0. 0.;
            0. 1. 0. 0.
            '''))
    return x, P

#Kalman Filter

Derived from Wikipedia

    See http://en.wikipedia.org/wiki/Kalman_filter

Look back over Kalman Introduction and Kalman Maths for an insight into how Kalman is operating.

The concept is:
* For a low computational cost
* Generate a state update/prediction
* Generate a measurement prediction from that state
* Calculate the difference between the predicted measurement and the actual measurement
* Adjust the state update/prediction, and repeat....

All done on normal probabilities - i.e. means and co-variances.  Effectively each key term is held as two parameters - a mean term and a covariance term for that mean.

To this filter, we supply the old state / variance and the new measurement value / variance and we take away a new state / variance.

Two key terms to watch are the process error estimate covariance term and the observation error estimate covariance term as their interaction influences the filter responsiveness.

In [0]:
def kalman(x, P, z, R, Q, F, H):
    '''
    Dynamic Parameters
    x: state estimate
    P: state estimate uncertainty covariance
    z: observation
    R: the covariance of the observation noise

    Additionally
    F: the state-transition model
    H: the observation model - maps true state space into the observed space
    Q: the covariance of the process noise

    return:
        updated and predicted new values for (x, P)
    '''
    # Update Step
    # Update x and P based on measurement m
    # distance between measured and current position-belief

    # Innovation (or measurement) pre-fit residual
    y = np.matrix(z).T - H * x

    # Innovation (or pre-fit residual) co-variance
    S = H * P * H.T + R

    # Optimal Kalman Gain
    try:
      inv = S.I
    except:
      inv = np.linalg.pinv(S)
    K = P * H.T * inv

    # Updated (a posteriori state estimate)
    x = x + K * y

    # Updated a posteriori estimate co-variance
    I = np.matrix(np.eye(F.shape[0]))
    P = (I - K*H)*P

    # Predict Step
    # Predict x and P
    x = F*x
    P = F*P*F.T + Q

    return x, P

#Demo

## Program Execution
For each file:
* get centre of detection (if any) and confidence from Yolo
* feed Kalman with these values
* Extract $x_{pos}$ and $y_{pos}$ from Kalman state term.
* Print original centre of detection
* Print filtered centre of detection

## Initialisation

### State Estimate Initialisation
We need to initialize the state estimate vector to something.  We'll initialise it to all zeros for reproducibility.


$x_{0} = \begin{bmatrix}
  0 \\
  0 \\
  0 \\
  0 \\
  \end{bmatrix}$

### State Estimate Error Covariance Initialisation
We need to initialize the state estimate error covariance matrix (estimated accuracy of state estimate).  We'll initialise it along the diagonal as being highly uncertain as shown here.

$P_{0} = \begin{bmatrix}
 100 && 0 && 0 && 0 \\
 0 && 100 && 0 && 0 \\
 0 && 0 && 100 && 0 \\
 0 && 0 && 0 && 100
 \end{bmatrix}$

Typically we initialize the $P$ matrix on the diagonal, with each component corresponding to the expected variance in the corresponding state term, i.e. how much deviation you might expect in the initialization of that state term.  If you have no insight into the behaviour of your model, this is a fairly typical way to get started while figuring out a better $P$ matrix initialisation.

### Observation Estimate Noise Covariance
And we'll set up a matrix to hold R, the measurement uncertainty (or more formally the estimated covariance of the observation noise.

 $R_{0} = \begin{bmatrix}
  0 && 0 \\
  0 && 0 
  \end{bmatrix}$

As we process each frame, we're going to adjust this term based on the confidence yolo has in its detection.

In [0]:
writer = None

def demo_kalman_2d():
    global writer
    # Initialise state estimate (x, y, x_dot, y_dot) to no position
    x = np.matrix('0. 0. 0. 0.').T
    # Initialise uncertainty to all highly uncertain
    P = np.matrix(np.eye(4))*100

    raw_centres = []        # a list of unfiltered centres
    filtered_centres = []   # a list of filtered centres

    R = np.zeros(shape=(2, 2))  # a shape to hold measurement uncertainty

    bb_lines, pngfiles = get_pngs_and_boxes()

    for pngfile in pngfiles:
        #print("handling .." + os.path.basename(pngfile))
        img = cv2.imread(pngfile)

        # Derive R from yolo confidence level in detection
        raw_centre, conf = parse_detections(bb_lines, pngfile, img)

        # Crudely derive R (the covariance of the observation noise). 
        # If yolo is confident we want a small
        # uncertainty. If yolo isn't confident, translate to
        # a large uncertainty.
        R *= 1/conf

        # Keep track of unfiltered bounding box centres - these will be
        # the basis of our Kalman
        raw_centres.append(raw_centre.astype(int))

        # reshape observation for Kalman - it expects
        # [x]
        # [y]
        # not [x,y]
        observed = raw_centre.reshape(1, 2)

        # Update the Kalman with state estimate, state covariance
        # observation and observation noise estimate
        x, P = kalman_2d(x, P, observed, R)

        # just track x and y values from Kalman state
        # (we just want to visualise positions)
        filtered_centre = x[:2]

        # Keep track of filtered x & y values
        filtered_centres.append(filtered_centre)

        # Pretty print a track of original centres
        # and filtered centres
        for i in range(1, len(raw_centres)):
            # print unfiltered
            cv2.line(img, tuple(raw_centres[i-1]), 
                     tuple(raw_centres[i]), (0, 255, 0), 20)

            # print filtered
            cv2.line(img, tuple(filtered_centres[i-1]), 
                     tuple(filtered_centres[i]), (255, 0, 0), 20)

        # Resize and show the image
        img = cv2.resize(img, (int(img.shape[1]/4), int(img.shape[0]/4)))

        # Build a frame of our output video
        if writer is None:
            # Initialize our video writer
            fourcc = cv2.VideoWriter_fourcc(*'VP80')
            writer = cv2.VideoWriter('video.webm', fourcc, 30, (img.shape[1], img.shape[0]), True)

        # Write the output frame to disk
        writer.write(img)

    # Release the file pointers

    writer.release()

demo_kalman_2d()


**Video**

Thia code plays the video we just made.

The Kalman Filtered track plays in green, the unfilterered track plays in blue.

As you can see, the Kalman Filtering has a role to play in predicting a reasonable guess for where the object might be while it is off-camera.

In [0]:
# Set this to 1 if video display
# is not working - works with chrome and firefox, not with safari
videoBodge = 0

def arrayShow (imageArray):
    ret, png = cv2.imencode('.png', imageArray)
    encoded = base64.b64encode(png)
    return Image(data=encoded.decode('ascii'))

if(videoBodge == 0):
    from IPython.display import HTML
    from base64 import b64encode
    webm = open('video.webm','rb').read()
    data_url = "data:video/webm;base64," + b64encode(webm).decode()
else:
    video = cv2.VideoCapture("video.webm")
    while(video.isOpened()):
        clear_output(wait=True)
        ret, frame = video.read()
        if(ret == False):
          break
        lines, columns, _ =  frame.shape
        img = arrayShow(frame)
        display(img)
        time.sleep(1)

In [0]:
# Display Video
HTML("""
<video width=200 controls>
      <source src="%s" type="video/webm">
</video>
""" % data_url)

# Conclusion

## Exercises
**Exercise 1**
Simulate occluding the object being detected - for example, only supply every second measurement update to the Kalman algorithm and observe the Kalman predictions.

**Exercise 2**
Again, similarly to the Bayes lab, vary the initial state covariance, process covariance and the measurement covariance relative to each other and observe how that affects the Kalman Filter's predictions.

**Advanced Exercise**
Think about how you might extend the model to account for acceleration in x and y.

## Takeaways
1. You've seen a Kalman Filter used for single object tracking
2. You've seen that a Kalman Filter can help deal with occlusions - i.e. in this example the object being tracked disappeared for a few frames and the Kalman continued to predict motion for it based on its model.
3. You've seen that a typical approach to writing a Kalman is to develop the core algorithm independent of the number of terms in the state variable and to specialise it for a particular model.

## Next Steps
1. We'll see the Kalman's strengths lie with predictable behaviour (typically referred to as a linear model) and we'll look at a derivitive technique (the Particle Filter) that attempts to improve the Kalman in the presence of less predictable behaviour (aka a non-linear model).